# STEP 1 - Making Basic PySC2 Agent

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. Runnning 'Agent code' on jupyter notebook 

In [3]:
# unfortunately, PySC2 uses Abseil, which treats python code as if its run like an app
# This does not play well with jupyter notebook
# So we will need to monkeypatch sys.argv


import sys
#sys.argv = ["python", "--map", "AbyssalReef"]
sys.argv = ["python", "--map", "Simple64"]

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS-IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Run an agent."""



'Run an agent.'

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import importlib
import threading

from absl import app
from absl import flags
from future.builtins import range  # pylint: disable=redefined-builtin

from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.lib import point_flag
from pysc2.lib import stopwatch

FLAGS = flags.FLAGS

# because of Abseil's horrible design for running code underneath Colabs
# We have to pull out this ugly hack from the hat
if "flags_defined" not in globals():
    flags.DEFINE_bool("render", True, "Whether to render with pygame.")
    point_flag.DEFINE_point("feature_screen_size", "84",
                            "Resolution for screen feature layers.")
    point_flag.DEFINE_point("feature_minimap_size", "64",
                            "Resolution for minimap feature layers.")
    point_flag.DEFINE_point("rgb_screen_size", None,
                            "Resolution for rendered screen.")
    point_flag.DEFINE_point("rgb_minimap_size", None,
                            "Resolution for rendered minimap.")
    flags.DEFINE_enum("action_space", None, sc2_env.ActionSpace._member_names_,  # pylint: disable=protected-access
                      "Which action space to use. Needed if you take both feature "
                      "and rgb observations.")
    flags.DEFINE_bool("use_feature_units", True,
                      "Whether to include feature units.")
    flags.DEFINE_bool("disable_fog", False, "Whether to disable Fog of War.")

    flags.DEFINE_integer("max_agent_steps", 0, "Total agent steps.")
    flags.DEFINE_integer("game_steps_per_episode", None, "Game steps per episode.")
    flags.DEFINE_integer("max_episodes", 0, "Total episodes.")
    flags.DEFINE_integer("step_mul", 8, "Game steps per agent step.")
    flags.DEFINE_float("fps", 22.4, "Frames per second to run the game.")

    #flags.DEFINE_string("agent", "sc2.agent.BasicAgent.ZergBasicAgent",
    #                    "Which agent to run, as a python path to an Agent class.")
    #flags.DEFINE_enum("agent_race", "zerg", sc2_env.Race._member_names_,  # pylint: disable=protected-access
    #                  "Agent 1's race.")
    flags.DEFINE_string("agent", "TerranBasicAgent",
                        "Which agent to run, as a python path to an Agent class.")
    flags.DEFINE_enum("agent_race", "terran", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 1's race.")

    flags.DEFINE_string("agent2", "Bot", "Second agent, either Bot or agent class.")
    flags.DEFINE_enum("agent2_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 2's race.")
    flags.DEFINE_enum("difficulty", "very_easy", sc2_env.Difficulty._member_names_,  # pylint: disable=protected-access
                      "If agent2 is a built-in Bot, it's strength.")

    flags.DEFINE_bool("profile", False, "Whether to turn on code profiling.")
    flags.DEFINE_bool("trace", False, "Whether to trace the code execution.")
    flags.DEFINE_integer("parallel", 1, "How many instances to run in parallel.")

    flags.DEFINE_bool("save_replay", True, "Whether to save a replay at the end.")

    flags.DEFINE_string("map", None, "Name of a map to use.")
    flags.mark_flag_as_required("map")

flags_defined = True

def run_thread(agent_classes, players, map_name, visualize):
  """Run one thread worth of the environment with agents."""
  with sc2_env.SC2Env(
      map_name=map_name,
      players=players,
      agent_interface_format=sc2_env.parse_agent_interface_format(
          feature_screen=FLAGS.feature_screen_size,
          feature_minimap=FLAGS.feature_minimap_size,
          rgb_screen=FLAGS.rgb_screen_size,
          rgb_minimap=FLAGS.rgb_minimap_size,
          action_space=FLAGS.action_space,
          use_feature_units=FLAGS.use_feature_units),
      step_mul=FLAGS.step_mul,
      game_steps_per_episode=FLAGS.game_steps_per_episode,
      disable_fog=FLAGS.disable_fog,
      visualize=visualize) as env:
    env = available_actions_printer.AvailableActionsPrinter(env)
    agents = [agent_cls() for agent_cls in agent_classes]
    run_loop.run_loop(agents, env, FLAGS.max_agent_steps, FLAGS.max_episodes)
    if FLAGS.save_replay:
      env.save_replay(agent_classes[0].__name__)

def main(unused_argv):
  """Run an agent."""
  #stopwatch.sw.enabled = FLAGS.profile or FLAGS.trace
  #stopwatch.sw.trace = FLAGS.trace

  map_inst = maps.get(FLAGS.map)

  agent_classes = []
  players = []

  #agent_module, agent_name = FLAGS.agent.rsplit(".", 1)
  #agent_cls = getattr(importlib.import_module(agent_module), agent_name)
  #agent_classes.append(agent_cls)
  agent_classes.append(TerranBasicAgent)
  players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent_race]))

  if map_inst.players >= 2:
    if FLAGS.agent2 == "Bot":
      players.append(sc2_env.Bot(sc2_env.Race[FLAGS.agent2_race],
                                 sc2_env.Difficulty[FLAGS.difficulty]))
    else:
      agent_module, agent_name = FLAGS.agent2.rsplit(".", 1)
      agent_cls = getattr(importlib.import_module(agent_module), agent_name)
      agent_classes.append(agent_cls)
      players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent2_race]))

  threads = []
  for _ in range(FLAGS.parallel - 1):
    t = threading.Thread(target=run_thread,
                         args=(agent_classes, players, FLAGS.map, False))
    threads.append(t)
    t.start()

  run_thread(agent_classes, players, FLAGS.map, FLAGS.render)

  for t in threads:
    t.join()

  if FLAGS.profile:
    pass
    #print(stopwatch.sw)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## 1. Creating a PySC2 Agent

In [5]:
from pysc2.agents import base_agent
from pysc2.env import sc2_env
from pysc2.lib import actions, features, units
from absl import app
import random
import time

In [6]:
class TerranBasicAgent(base_agent.BaseAgent):
  def __init__(self):
    super(TerranBasicAgent, self).__init__()

  def step(self, obs):
    super(TerranBasicAgent, self).step(obs)

    time.sleep(0.1)

    return actions.FUNCTIONS.no_op()

# 4. Controlling Factory

In [99]:
class TerranBasicAgent(base_agent.BaseAgent):
    def __init__(self):
        super(TerranBasicAgent, self).__init__()
        
        self.base_top_left = None
        self.supply_depot_built = False
        self.barracks_built = False
        self.barracks_rallied = False
        self.army_rallied = False

    def transformLocation(self, x, x_distance, y, y_distance):
        if not self.base_top_left:
            return [abs(x - x_distance), abs(y - y_distance)]
        
        return [abs(x + x_distance), abs(y + y_distance)]

    def getMeanLocation(self, unitList):
        sum_x = 0
        sum_y = 0
        for unit in unitList:
            sum_x += unit.x
            sum_y += unit.y
        mean_x = sum_x / len(unitList)
        mean_y = sum_y / len(unitList)
        
        return [mean_x, mean_y]
    
    def getOneLocation(self, unitList):
        x = 0
        y = 0
        for unit in unitList:
            x += unit.x
            y += unit.y
            break
        
        return [x, y]
    
    def unit_type_is_selected(self, obs, unit_type):
        if (len(obs.observation.single_select) > 0 and
            obs.observation.single_select[0].unit_type == unit_type):
              return True

        if (len(obs.observation.multi_select) > 0 and
            obs.observation.multi_select[0].unit_type == unit_type):
              return True

        return False

    def get_units_by_type(self, obs, unit_type):
        return [unit for unit in obs.observation.feature_units
                if unit.unit_type == unit_type]

    def can_do(self, obs, action):
        return action in obs.observation.available_actions

    def step(self, obs):
        super(TerranBasicAgent, self).step(obs)
        
        time.sleep(0.05)
        
        if obs.first():
            self.base_top_left = None
            self.supply_depot_built = False
            self.barracks_built = False
            self.barracks_rallied = False
            self.refinery_built = False
            self.refinery_rallied = False
            self.factorys_built = False
            self.factorys_rallied = False
            self.army_rallied = False
            
            player_y, player_x = (obs.observation.feature_minimap.player_relative == features.PlayerRelative.SELF).nonzero()
            self.base_top_left = 1 if player_y.any() and player_y.mean() <= 31 else 0
            
        
        ################### refinery 제작 ####################################################
        if not self.refinery_built:            
            if self.unit_type_is_selected(obs, units.Terran.SCV):
                if self.can_do(obs, actions.FUNCTIONS.Build_Refinery_screen.id):
                    gas_pos = self.get_units_by_type(obs, units.Neutral.VespeneGeyser)
                    if len(gas_pos) > 0:
                        x,y = self.getOneLocation(gas_pos)
                        target = [x,y]
                        self.refinery_built = True

                        return actions.FUNCTIONS.Build_Refinery_screen("now", target)
            scvs = self.get_units_by_type(obs, units.Terran.SCV)
            if len(scvs) > 0:
                scv = random.choice(scvs)
                return actions.FUNCTIONS.select_point("select", (scv.x,
                                                                          scv.y))

        ######################################################################################
        
        if not self.supply_depot_built:
            print('supply 제작')
            if self.unit_type_is_selected(obs, units.Terran.SCV):
                if self.can_do(obs, actions.FUNCTIONS.Build_SupplyDepot_screen.id):
                    ccs = self.get_units_by_type(obs, units.Terran.CommandCenter)
                    if len(ccs) > 0:
                        mean_x, mean_y = self.getMeanLocation(ccs)
                        target = self.transformLocation(int(mean_x), 0, int(mean_y), 20)
                        self.supply_depot_built = True

                        return actions.FUNCTIONS.Build_SupplyDepot_screen("now", target)
            scvs = self.get_units_by_type(obs, units.Terran.SCV)
            if len(scvs) > 0:
                scv = random.choice(scvs)
                return actions.FUNCTIONS.select_point("select", (scv.x,
                                                                          scv.y))
        elif not self.barracks_built:
            print('barracks 제작')
            if self.unit_type_is_selected(obs, units.Terran.SCV):
                if self.can_do(obs, actions.FUNCTIONS.Build_Barracks_screen.id):
                    ccs = self.get_units_by_type(obs, units.Terran.CommandCenter)
                    if len(ccs) > 0:
                        mean_x, mean_y = self.getMeanLocation(ccs)
                        target = self.transformLocation(int(mean_x), 20, int(mean_y), 0)
                        self.barracks_built = True

                        return actions.FUNCTIONS.Build_Barracks_screen("now", target)
            scvs = self.get_units_by_type(obs, units.Terran.SCV)
            if len(scvs) > 0:
                scv = random.choice(scvs)
                return actions.FUNCTIONS.select_point("select", (scv.x,
                                                                          scv.y))
        
        elif not self.barracks_rallied:
            print('barracks rallied')
            if self.unit_type_is_selected(obs, units.Terran.Barracks):
                self.barracks_rallied = True
                
                if self.base_top_left:
                    return actions.FUNCTIONS.Rally_Units_minimap("now", [29, 21])
                else:
                    return actions.FUNCTIONS.Rally_Units_minimap("now", [29, 46])
            barracks = self.get_units_by_type(obs, units.Terran.Barracks)
            if len(barracks) > 0:
                barrack = random.choice(barracks)
                return actions.FUNCTIONS.select_point("select", (barrack.x,
                                                                          barrack.y))
            
            
        elif obs.observation.player.food_cap - obs.observation.player.food_used > 8:
            print('marine 제작')
            if self.can_do(obs, actions.FUNCTIONS.Train_Marine_quick.id):
                return actions.FUNCTIONS.Train_Marine_quick("queued")
        
        
        ####################### Factory 건설 ########################################
        
        elif not self.factorys_built:
            print('factory 제작')
            if self.unit_type_is_selected(obs, units.Terran.SCV):
                if self.can_do(obs, actions.FUNCTIONS.Build_Factory_screen.id):
                    brs = self.get_units_by_type(obs, units.Terran.Barracks)
                    if len(brs) > 0:
                        mean_x, mean_y = self.getMeanLocation(brs)
                        target = self.transformLocation(int(mean_x), 10, int(mean_y), 10)
                        self.factorys_built = True

                        return actions.FUNCTIONS.Build_Factory_screen("now", target)
            scvs = self.get_units_by_type(obs, units.Terran.SCV)
            if len(scvs) > 0:
                scv = random.choice(scvs)
                return actions.FUNCTIONS.select_point("select", (scv.x,
                                                                          scv.y))
        
        elif not self.factorys_rallied:
            print('factory rallied')
            if self.unit_type_is_selected(obs, units.Terran.Factory):
                self.factorys_rallied = True
                
                if self.base_top_left:
                    return actions.FUNCTIONS.Rally_Units_minimap("now", [29, 21])
                else:
                    return actions.FUNCTIONS.Rally_Units_minimap("now", [29, 46])
                
            factorys = self.get_units_by_type(obs, units.Terran.Factory)
            if len(factorys) > 0:
                factory = random.choice(factorys)
                return actions.FUNCTIONS.select_point("select", (factory.x,
                                                                          factory.y))
        
        elif obs.observation.player.food_cap - obs.observation.player.food_used:
            print('hellion 제작')
            if self.can_do(obs, actions.FUNCTIONS.Train_Hellion_quick.id):
                return actions.FUNCTIONS.Train_Hellion_quick("queued")
        
        
        
        if obs.observation.player.food_used > 30:
            print('공격시작')
            if self.can_do(obs, actions.FUNCTIONS.Attack_minimap.id):
                self.army_rallied = True
                
                if self.base_top_left:
                    return actions.FUNCTIONS.Attack_minimap("now", [39, 45])
                else:
                    return actions.FUNCTIONS.Attack_minimap("now", [21, 24])

            if self.can_do(obs, actions.FUNCTIONS.select_army.id):
                return actions.FUNCTIONS.select_army("select")
            
        elif obs.observation.player.food_cap - obs.observation.player.food_used < 3:
            print('supply 제작')
            if self.unit_type_is_selected(obs, units.Terran.SCV):
                if self.can_do(obs, actions.FUNCTIONS.Build_SupplyDepot_screen.id):
                    sup = self.get_units_by_type(obs, units.Terran.SupplyDepot)
                    if len(sup) > 0:
                        mean_x, mean_y = self.getMeanLocation(sup)
                        target = self.transformLocation(int(mean_x), 25, int(mean_y), 25)
                        self.supply_depot_built = True

                        return actions.FUNCTIONS.Build_SupplyDepot_screen("now", target)
            scvs = self.get_units_by_type(obs, units.Terran.SCV)
            if len(scvs) > 0:
                scv = random.choice(scvs)
                return actions.FUNCTIONS.select_point("select", (scv.x,
                                                                          scv.y))
        else:
            factorys = self.get_units_by_type(obs, units.Terran.Factory)
            if len(factorys) > 0:
                factory = random.choice(factorys)
                return actions.FUNCTIONS.select_point("select", (factory.x,
                                                                          factory.y))
            
        if obs.observation.player.food_cap - obs.observation.player.food_used > 8:
            print('marine 제작')
            if self.can_do(obs, actions.FUNCTIONS.Train_Marine_quick.id):
                return actions.FUNCTIONS.Train_Marine_quick("queued")
        
        return actions.FUNCTIONS.no_op()

### [run code]

In [101]:
if __name__ == "__main__":
  app.run(main)

I0826 03:05:26.529365 4645408192 sc_process.py:135] Launching SC2: /Applications/StarCraft II/Versions/Base81102/SC2.app/Contents/MacOS/SC2 -listen 127.0.0.1 -port 21992 -dataDir /Applications/StarCraft II/ -tempDir /var/folders/r1/x6k135_915z463fc7lc4hkp40000gn/T/sc-ch901nut/ -displayMode 0 -windowwidth 640 -windowheight 480 -windowx 50 -windowy 50
I0826 03:05:26.584736 4645408192 remote_controller.py:167] Connecting to: ws://127.0.0.1:21992/sc2api, attempt: 0, running: True
I0826 03:05:27.593104 4645408192 remote_controller.py:167] Connecting to: ws://127.0.0.1:21992/sc2api, attempt: 1, running: True
I0826 03:05:28.596437 4645408192 remote_controller.py:167] Connecting to: ws://127.0.0.1:21992/sc2api, attempt: 2, running: True
I0826 03:05:29.602042 4645408192 remote_controller.py:167] Connecting to: ws://127.0.0.1:21992/sc2api, attempt: 3, running: True
I0826 03:05:30.604841 4645408192 remote_controller.py:167] Connecting to: ws://127.0.0.1:21992/sc2api, attempt: 4, running: True
I08

   0/no_op                                              ()
   1/move_camera                                        (1/minimap [64, 64])
   2/select_point                                       (6/select_point_act [4]; 0/screen [84, 84])
   3/select_rect                                        (7/select_add [2]; 0/screen [84, 84]; 2/screen2 [84, 84])
   4/select_control_group                               (4/control_group_act [5]; 5/control_group_id [10])
 453/Stop_quick                                         (3/queued [2])
 230/Effect_Spray_screen                                (3/queued [2]; 0/screen [84, 84])
 549/Effect_Spray_minimap                               (3/queued [2]; 1/minimap [64, 64])
 264/Harvest_Gather_screen                              (3/queued [2]; 0/screen [84, 84])
 451/Smart_screen                                       (3/queued [2]; 0/screen [84, 84])
 452/Smart_minimap                                      (3/queued [2]; 1/minimap [64, 64])
 331/Move_screen    

marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
marine 제작
factory 제작
  43/Build_Bunker_screen                                (3/queued [2]; 0/screen [84, 84])
  44/Build_CommandCenter_screen                         (3/queued [2]; 0/screen [84, 84])
  58/Build_GhostAcademy_screen                   

hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
su

hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
su

...

hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
supply 제작
hellion 제작
su

I0826 03:11:01.073604 4645408192 sc2_env.py:752] Environment Close


marine 제작
Took 306.424 seconds for 3833 steps: 12.509 fps


I0826 03:11:01.383404 4645408192 sc_process.py:232] Shutdown gracefully.
I0826 03:11:01.384503 4645408192 sc_process.py:210] Shutdown with return code: -15


ValueError: Argument is out of range for 2/select_point (6/select_point_act [4]; 0/screen [0, 0]), got: [[<SelectPointAct.select: 0>], [19, -25]]